In [ ]:
checkpoint = widgets.Dropdown(options=[('свежая', 1), ('стабильная', 2)], value=1, description='версия A1111:').add_class("checkpoint")
gdrive = widgets.Checkbox(value=False, description='подключить гуглодиск').add_class("googledisk_mount")
gdrive_models = widgets.Checkbox(value=False, description='подтянуть модели с гуглодиска').add_class("googledisk_symlink")
gfolder = widgets.Text(value="/content/gdrive/MyDrive/SD/", placeholder='там где лежат папки с моделями и т.п.').add_class("gfolder")
gdrive_only = widgets.Checkbox(value=False, description='модели только с гуглодиска').add_class("googledisk_only")
xformers = widgets.Checkbox(value=True, description='xformers').add_class("xformers")
pix2pix = widgets.Checkbox(value=False, description='instruct pix2pix').add_class("pix2pix")
controlnet = widgets.Checkbox(value=False, description='ControlNet').add_class("controlnet")
extentions = widgets.Checkbox(value=False, description='дополнения').add_class("extentions")
extentions_mix = widgets.Checkbox(value=False, description='дополнения для слияние моделей').add_class("extentions_mix")
embeddings = widgets.Checkbox(value=False, description='внедрения').add_class("embeddings")
hypernet = widgets.Checkbox(value=False, description='гиперсети').add_class("hypernet")
booru = widgets.Checkbox(value=False, description='DeepDanbooru').add_class("booru")
no_half_vae = widgets.Checkbox(value=True, description='--no-half-vae').add_class("no_half_vae")
starton = widgets.Checkbox(value=True, description='запустить сразу').add_class("starton")
gd_set = widgets.HBox([gdrive, gdrive_models, gfolder, gdrive_only]).add_class("gd_set")
eng = widgets.Checkbox(value=False, description='англоязычный интерфейс').add_class("eng")
ngrok_token = widgets.Text(value="", placeholder='тут можно указать свой токен ngrok').add_class("ngrok_token")
additions = widgets.HBox([controlnet, pix2pix, extentions, extentions_mix, embeddings, hypernet, booru]).add_class("additions")
webuiargs = widgets.HBox([xformers, no_half_vae, starton, eng, ngrok_token]).add_class("args")
apply = widgets.Button(description='продолжить ❯').add_class("apply")
model_header = HTML('<h2 id="model_header">выбор моделей</h2><p id="model_description"><a href="https://teletype.in/@stablediffusion/models_for_colab" target="_blank">описания моделей, ключи для их вызова</p>')


# Примонтировать модели, эмбендинги, лоры, и прочее с гугл диска. Версия THREADING + RE. Рекурсивная. РегистроНЕчувствительная.
def ultimate_gdrive_symlink():
  if gdrive_models.value == True:
    def process_dir(src_dir, dest_dir):
        if os.path.isdir(src_dir):
            dest_dir_exists = False
            for dirname_dest in os.listdir(destination_dir):
                if re.match(f'^{os.path.basename(src_dir)}$', dirname_dest, re.IGNORECASE):
                    dest_dir = os.path.join(destination_dir, dirname_dest)
                    dest_dir_exists = True
                    break
            if not dest_dir_exists:
              try:
                os.makedirs(dest_dir)
              except:
                pass
            for item in os.listdir(src_dir):
                src_item = os.path.join(src_dir, item)
                dest_item = os.path.join(dest_dir, item)
                if os.path.isdir(src_item):
                    process_dir(src_item, dest_item)
                else:
                    try:
                        os.symlink(src_item, dest_item)
                    except FileExistsError:
                        pass
    source_dir = gfolder.value
    if not source_dir.startswith("/content/gdrive/MyDrive/"): # Если исходный путь не начинается с "/content/gdrive/MyDrive/", фиксим то дополнив его. Обратно совместимо вписанием полного пути.
        source_dir = os.path.join("/content/gdrive/MyDrive/", source_dir)
    destination_dir = "/content/stable-diffusion-webui/"
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
    drive.mount('/content/gdrive', force_remount=True)
    print('гуглодиск примонтирован')
    #Используем регулярное выражение для поиска имени исходной папки, фикся регистр
    pattern = re.compile(re.escape(os.path.basename(source_dir)), re.IGNORECASE)
    # Находим все папки в исходной папке
    directories = [d for d in os.listdir(os.path.dirname(source_dir)) if os.path.isdir(os.path.join(os.path.dirname(source_dir), d))]
    # ищем нужную
    match = next((d for d in directories if pattern.search(d)), None)
    # Если нашли, обновляем source_dir
    if match:
        source_dir = os.path.join(os.path.dirname(source_dir), match)
    if not os.path.exists(source_dir):
        print("source_dir не существует или указана неверно")
    else:
      threads = [] #попытка в потоки
      for dirname in os.listdir(source_dir):
            src_dir = os.path.join(source_dir, dirname)
            dest_dir = os.path.join(destination_dir, dirname)
            thread = threading.Thread(target=process_dir, args=(src_dir, dest_dir))
            threads.append(thread)
            thread.start()
      for thread in threads:
            thread.join()
  if gdrive_models.value == False:
    None

def lora_to_extention():
  lpath = "/content/stable-diffusion-webui/models/Lora"
  extpath = "/content/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora"
  extpath_dev2 = "/content/stable-diffusion-webui/extensions/sd-webui-additional-networks-dev2/models/lora"
  if not os.path.exists(extpath):
    os.makedirs(extpath)
  if not os.path.exists(extpath_dev2):
    os.makedirs(extpath_dev2)
  for l in os.listdir(lpath):
    try:
      os.symlink(f"{lpath}/{l}", f"{extpath}/{l}")
    except FileExistsError:
      pass
  for l in os.listdir(lpath):
    try:
      os.symlink(f"{lpath}/{l}", f"{extpath_dev2}/{l}")
    except FileExistsError:
      pass

#показать/спрятать указание папки на гдиске с моделями
def gd_path(change):
  if gdrive_models.value == True:
    gfolder.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
  if gdrive_models.value == False:
    gfolder.layout.display = 'none'
    gdrive_only.layout.display = 'none'

# примонтирование диска и показать/спрятать опции гдиска
def gdrive_mount(change):
  if gdrive.value == True:
    gdrive_models.layout.display = 'flex'
    gdrive_only.layout.display = 'flex'
    try:
      drive.mount('/content/gdrive', force_remount=True)
      print('гуглодиск примонтирован')
    except:
      print("пользователь отказался")
  if gdrive.value == False:
    print("гуглодиск не примонтирован")
    gdrive_models.layout.display = 'none'
    gdrive_only.layout.display = 'none'
    gfolder.layout.display = 'none'
    gdrive_models.value = False
    gfolder.value = ""
    gdrive_only.value = False

#показать/спрятать виджет моделей (если модели только с гдиска - виджет не нужен)
def only_gdrive(b):
  if gdrive_only.value == True:
    tab.layout.display = 'none'
    search_and_button.layout.display = 'none'
    display(HTML('<style>#model_header,#model_description{display:none}'))
  if gdrive_only.value == False:
    tab.layout.display = 'flex'
    search_and_button.layout.display = 'flex'
    display(HTML('<style>#model_header,#model_description{display:block}'))

# установка вебуи и основных зависимостей, правки
def A1111_webui_install():
  time.sleep(1.5)
  #!rm -rf /usr/local/lib/python3.8/dist-packages/scipy /usr/local/lib/python3.8/dist-packages/scipy-1.7.3.dist-info /usr/local/lib/python3.8/dist-packages/scipy.libs
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui/
  !cd /content/stable-diffusion-webui/
  if checkpoint_value == "new":
    %cd /content/stable-diffusion-webui
    !git checkout $commit_new
  if checkpoint_value == "legacy":
    %cd /content/stable-diffusion-webui
    !git checkout $commit_stable
  # установка репозиториев для вебуи
  with open('/content/stable-diffusion-webui/launch.py', 'r') as fp:
    lines = fp.readlines()
    for line in lines:
      if line.find('STABLE_DIFFUSION_COMMIT_HASH') != -1:
        stable_diffusion_commit_hash = line.split('"')[1]
      if line.find('TAMING_TRANSFORMERS_COMMIT_HASH') != -1:
        taming_transformers_commit_hash = line.split('"')[1]
      if line.find('K_DIFFUSION_COMMIT_HASH') != -1:
        k_diffusion_commit_hash = line.split('"')[1]
      if line.find('CODEFORMER_COMMIT_HASH') != -1:
        codeformer_commit_hash = line.split('"')[1]
      if line.find('BLIP_COMMIT_HASH') != -1:
        blip_commit_hash = line.split('"')[1]
  reps = [
      f"git clone https://github.com/Stability-AI/stablediffusion.git /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && cd /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai && git checkout {stable_diffusion_commit_hash}",
      f"git clone https://github.com/CompVis/taming-transformers.git /content/stable-diffusion-webui/repositories/taming-transformers && cd /content/stable-diffusion-webui/repositories/taming-transformers && git checkout {taming_transformers_commit_hash}",
      f"git clone https://github.com/crowsonkb/k-diffusion.git /content/stable-diffusion-webui/repositories/k-diffusion && cd /content/stable-diffusion-webui/repositories/k-diffusion && git checkout {k_diffusion_commit_hash}",
      f"git clone https://github.com/sczhou/CodeFormer.git /content/stable-diffusion-webui/repositories/CodeFormer && cd /content/stable-diffusion-webui/repositories/CodeFormer && git checkout {codeformer_commit_hash}",
      f"git clone https://github.com/salesforce/BLIP.git /content/stable-diffusion-webui/repositories/BLIP && cd /content/stable-diffusion-webui/repositories/BLIP && git checkout {blip_commit_hash}",
      "git clone https://github.com/isl-org/MiDaS.git /content/stable-diffusion-webui/repositories/midas",
      "wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/bin/cloudflared  && chmod +x /usr/bin/cloudflared",
      ]
  def reps_inst(rep):
    !$rep
  with ThreadPoolExecutor(max_workers=len(reps)) as executor:
    futures = [executor.submit(reps_inst, rep) for rep in reps]
    for future in as_completed(futures):
        future.result()
  os.environ["SAFETENSORS_FAST_GPU"]='1'
  !cd /content/stable-diffusion-webui
  !COMMANDLINE_ARGS="--exit" REQS_FILE="requirements.txt" python launch.py
  !rm /content/stable-diffusion-webui/modules/ui_tempdir.py
  !$dl_gh'ui-tempdir.py' -O /content/stable-diffusion-webui/modules/ui_tempdir.py
  #!git clone https://github.com/Stability-AI/stablediffusion
  #!rm -rf /content/stablediffusion/.git /content/stablediffusion/.gitignore /content/stablediffusion/LICENSE /content/stablediffusion/LICENSE-MODEL /content/stablediffusion/README.md /content/stablediffusion/modelcard.md /content/stablediffusion/setup.py /content/stablediffusion/requirements.txt /content/sample_data 
  #distutils.dir_util.copy_tree('/content/stablediffusion', '/content/stable-diffusion-webui')
  #!rm -rf /content/stablediffusion
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/stable-diffusion-webui/webui.py
  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/stable-diffusion-webui/modules/sd_models.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/stable-diffusion-webui/modules/extras.py
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' /content/stable-diffusion-webui/modules/shared.py
  with open("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py") as r: text = r.read().replace('return get_obj_from_str(config["target"])(**config.get("params", dict()))', 'return get_obj_from_str(config["target"])(**config.get("params", dict())).cuda()')
  with open("/content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py", "w") as w: w.write(text)
  clear()

# скачивание модели и дополнения instruct-pix2pix
def instruct_pix2pix():
  time.sleep(1.5)
  if pix2pix.value == True:
    if checkpoint_value == "legacy":
      print("\033[01;38;05;208mInstruct Pix2Pix модель и расширение не будут установленны,\nпотому что ты не выбрал свежую версию A1111!\033[0m")
      cou.eval_js(f'new Audio("{gh}pixwarn.mp3").play()')
    if checkpoint_value == "new":
      instructs = [
          "git clone https://github.com/Klace/stable-diffusion-webui-instruct-pix2pix /content/stable-diffusion-webui/extensions/stable-diffusion-webui-instruct-pix2pix",
          f"{dl} {sf}instruct-pix2pix.safetensors -c -P /content/stable-diffusion-webui/models/Stable-diffusion/",
          #"ln -s /content/stable-diffusion-webui/models/VAE/vae-ft-mse.ckpt /content/stable-diffusion-webui/models/Stable-diffusion/instruct_pix2pix.vae.pt"
        ]
      def pix2pix_ext(instruct):
          !$instruct
      with ThreadPoolExecutor(max_workers=len(instructs)) as executor:
          futures = [executor.submit(pix2pix_ext, instruct) for instruct in instructs]
          for future in as_completed(futures):
              future.result()
  clear()
  if pix2pix.value == False:
    None


# скачивание моделей и дополнения ControlNet
def ControlNet_install():
  time.sleep(1.5)
  if controlnet.value == True:
    !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
    with open("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts/controlnet.py") as r: text = r.read().replace('# "openpose_hand": openpose_hand,', '"openpose_hand": openpose_hand,')
    with open("/content/stable-diffusion-webui/extensions/sd-webui-controlnet/scripts/controlnet.py", "w") as w: w.write(text)
    !rm /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/cldm_v15.yaml
    dlcn = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models "
    controlnets = [
        "pip -q install svglib",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/cldm_v15.yaml",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors",
        f"{dlcn} https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/dpt_hybrid-midas-501f0c75.pt -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/midas",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_large_512_fp32.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/mlsd_tiny_512_fp32.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/mlsd",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/upernet_global_small.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/uniformer",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/hand_pose_model.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/body_pose_model.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/openpose",
        f"{wget} https://huggingface.co/lllyasviel/ControlNet/resolve/main/annotator/ckpts/network-bsds500.pth -P /content/stable-diffusion-webui/extensions/sd-webui-controlnet/annotator/hed",
      ]
    def controlnet_ext(controlnet):
        !$controlnet
    with ThreadPoolExecutor(max_workers=len(controlnets)) as executor:
        futures = [executor.submit(controlnet_ext, controlnet) for controlnet in controlnets]
        for future in as_completed(futures):
            future.result()
  clear()
  if controlnet.value == False:
    None

# дополнение clip-interrogator
def interrogate_clip_install():
  time.sleep(1.5)
  !git clone --depth 1 https://github.com/pharmapsychotic/clip-interrogator-ext /content/stable-diffusion-webui/extensions/clip-interrogator-ext
  ipath = "/content/stable-diffusion-webui/models/clip-interrogator"
  dl = f"{wget} -P {ipath} "
  clips = [
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_artists.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_flavors.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_mediums.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_movements.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_negative.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/RN50_openai_trendings.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_artists.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_flavors.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_mediums.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_movements.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_negative.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-H-14_laion2b_s32b_b79k_trendings.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_artists.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_flavors.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_mediums.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_movements.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_negative.safetensors",
      f"{dl} https://huggingface.co/pharma/ci-preprocess/resolve/main/ViT-L-14_openai_trendings.safetensors",
      "pip -q install clip-interrogator==0.5.4",
      ]
  def clips_inst(clip):
    !$clip
  with ThreadPoolExecutor(max_workers=len(clips)) as executor:
    futures = [executor.submit(clips_inst, clip) for clip in clips]
    for future in as_completed(futures):
        future.result()
    clear()

# обязательные дополнения
def extentions_builtin():
  time.sleep(1.5)
  #таймер, описания моделей и апскейлер, замена лиц, пропорции картинки, генератор промптов
  os.makedirs('/content/stable-diffusion-webui/static', exist_ok = True)
  builtins = [
    f"wget -q {gh}colab-timer.js -P /content/stable-diffusion-webui/extensions-builtin/timer/javascript/",
    "git clone https://github.com/PR0LAPSE/colab_models_list /content/stable-diffusion-webui/extensions-builtin/colab_models_list",
    "git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimate-upscale-for-automatic1111",
    "wget -q https://github.com/kohya-ss/sd-webui-additional-networks/archive/refs/heads/dev2.zip",
    "git clone https://github.com/kex0/batch-face-swap /content/stable-diffusion-webui/extensions/batch-face-swap",
    "pip -q install mediapipe",
    "git clone https://github.com/alemelis/sd-webui-ar /content/stable-diffusion-webui/extensions/sd-webui-ar",
    "git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-promptgen /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen",
  ]
  def ext_builtin_inst(builtin):
    !$builtin
  with ThreadPoolExecutor(max_workers=len(builtins)) as executor:
    futures = [executor.submit(ext_builtin_inst, builtin) for builtin in builtins]
    for future in as_completed(futures):
        future.result()
  !unzip -n -q dev2.zip -d /content/stable-diffusion-webui/extensions && rm dev2.zip
  !echo -n {launching} > /content/stable-diffusion-webui/static/launch.txt
  #скрипты
  scrurls = [
      "https://github.com/GRMrGecko/stable-diffusion-webui-automatic/raw/advanced_matrix/scripts/advanced_prompt_matrix.py",
      "https://github.com/feffy380/prompt-morph/raw/master/prompt_morph.py",
      "https://github.com/yownas/shift-attention/raw/main/scripts/shift_attention.py",
      "https://github.com/Filarius/stable-diffusion-webui/raw/master/scripts/vid2vid.py",
      "https://github.com/Animator-Anon/Animator/raw/main/animation.py",
      "https://github.com/Animator-Anon/Animator/raw/main/animation_v6.py",
      "https://github.com/yownas/seed_travel/raw/main/scripts/seed_travel.py",
      "https://github.com/memes-forever/Stable-diffusion-webui-video/raw/main/videos.py",
      "https://github.com/dfaker/embedding-to-png-script/raw/main/embedding_to_png.py",
      "https://github.com/dfaker/stable-diffusion-webui-cv2-external-masking-script/raw/main/external_masking.py",
  ]
  def download_sdscripts(scrurl):
    !$dl $scrurl -P /content/stable-diffusion-webui/scripts
  with ThreadPoolExecutor(max_workers=len(scrurls)) as executor:
    futures = [executor.submit(download_sdscripts, scrurl) for scrurl in scrurls]
    for future in as_completed(futures):
        future.result()
  clear()

#скачивание интеррогейтов и языковых моделей для составления промпта (быстрее чем через вебуи)
def misc_models():
  time.sleep(1.5)
  miscmodels = [
  "git clone https://huggingface.co/AUTOMATIC/promptgen-lexart /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen/models/promptgen-lexart",
  "git clone https://huggingface.co/AUTOMATIC/promptgen-majinai-safe /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen/models/promptgen-majinai-safe",
  "git clone https://huggingface.co/AUTOMATIC/promptgen-majinai-unsafe /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen/models/promptgen-majinai-unsafe",
  "git clone https://huggingface.co/succinctly/text2image-prompt-generator /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen/models/text2image-prompt-generator",
  "git clone https://huggingface.co/microsoft/Promptist /content/stable-diffusion-webui/extensions/stable-diffusion-webui-promptgen/models/Promptist",
  f"{wget} https://github.com/AUTOMATIC1111/TorchDeepDanbooru/releases/download/v1/model-resnet_custom_v3.pt -P /content/stable-diffusion-webui/models/torch_deepdanbooru",
  f"{wget} https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_caption_capfilt_large.pth -P /content/stable-diffusion-webui/models/BLIP",
  f"wget -q {gh}artists.txt -P /content/stable-diffusion-webui/interrogate",
  f"wget -q {gh}flavors.txt -P /content/stable-diffusion-webui/interrogate",
  f"wget -q {gh}mediums.txt -P /content/stable-diffusion-webui/interrogate",
  f"wget -q {gh}movements.txt -P /content/stable-diffusion-webui/interrogate",
  ]
  def misc_models_inst(miscmodel):
    !$miscmodel
  with ThreadPoolExecutor(max_workers=len(miscmodels)) as executor:
    futures = [executor.submit(misc_models_inst, miscmodel) for miscmodel in miscmodels]
    for future in as_completed(futures):
        future.result()
  clear()


# скачивание дополнений/скриптов
def extentions_install():
  time.sleep(1.5)
  if extentions.value == True:
    sdextentions = [
    "git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-aesthetic-gradients /content/stable-diffusion-webui/extensions/aesthetic-gradients",
    "git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-wildcards /content/stable-diffusion-webui/extensions/wildcards",
    "git clone https://github.com/yfszzx/stable-diffusion-webui-inspiration /content/stable-diffusion-webui/extensions/inspiration",
    "git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-tokenizer /content/stable-diffusion-webui/extensions/tokenizer",
    "git clone https://github.com/dfaker/SD-latent-mirroring /content/stable-diffusion-webui/extensions/latent-mirroring",
    "git clone https://github.com/mix1009/model-keyword /content/stable-diffusion-webui/extensions/model-keyword",
    "git clone https://github.com/OedoSoldier/enhanced-img2img /content/stable-diffusion-webui/extensions/enhanced-img2img",
    ]
    def ext_sdextention_inst(sdextention):
      !$sdextention
    with ThreadPoolExecutor(max_workers=len(sdextentions)) as executor:
      futures = [executor.submit(ext_sdextention_inst, sdextention) for sdextention in sdextentions]
      for future in as_completed(futures):
          future.result()
  clear()
  if extentions.value == False:
    None

def extentions_mix_install():
  time.sleep(1.5)
  if extentions_mix.value == True:
    mixextentions = [
    "git clone https://github.com/arenatemp/stable-diffusion-webui-model-toolkit /content/stable-diffusion-webui/extensions/stable-diffusion-webui-model-toolkit",
    "git clone https://github.com/bbc-mc/sdweb-merge-block-weighted-gui /content/stable-diffusion-webui/extensions/sdweb-merge-block-weighted-gui",
    "git clone https://github.com/bbc-mc/sdweb-merge-board /content/stable-diffusion-webui/extensions/sdweb-merge-board",
    ]
    def mixextention_inst(mixextention):
      !$mixextention
    with ThreadPoolExecutor(max_workers=len(mixextentions)) as executor:
      futures = [executor.submit(mixextention_inst, mixextention) for mixextention in mixextentions]
      for future in as_completed(futures):
          future.result()
  clear()
  if extentions_mix.value == False:
    None

def upscalers():
  time.sleep(1.5)
  upscmodels = [
  "wget -q https://huggingface.co/utnah/LDSR/resolve/main/project.yaml -P /content/stable-diffusion-webui/models/LDSR",
  "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition https://huggingface.co/utnah/LDSR/resolve/main/model.safetensors -P /content/stable-diffusion-webui/models/LDSR",
  f"{dl} {esrgan}4x-UltraSharp.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}4x-AnimeSharp.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}4x_foolhardy_Remacri.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}4xESRGAN.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}4x_Nickelback_70000G.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}lollypop.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}1x_artifacts_jpg_20_40_alsa.pth -P /content/stable-diffusion-webui/models/ESRGAN",
  f"{dl} {esrgan}RealESRGAN_x4plus_anime_6B.pth -P /content/stable-diffusion-webui/models/RealESRGAN",
  ]
  def upscmodels_inst(upscmodel):
    !$upscmodel
  with ThreadPoolExecutor(max_workers=len(upscmodels)) as executor:
    futures = [executor.submit(upscmodels_inst, upscmodel) for upscmodel in upscmodels]
    for future in as_completed(futures):
        future.result()
  #!cp -a /content/stable-diffusion-webui/repositories/latent-diffusion* /content/stable-diffusion-webui/modules
  clear()

def emdeddings_install():
  time.sleep(1.5)
  if embeddings.value == True:
    !$dl $pt'embeddings' -O e.zip
    !7z -bsp1 -bso0 -y -bt x e.zip -o"/content/stable-diffusion-webui/embeddings"
    !rm e.zip
    clear()
  if embeddings.value == False:
    None
def emdeddings_for_negative():
  time.sleep(1.5)
  !$dl $pt'neg_embeddings' -O n.zip
  !7z -bsp1 -bso0 -y -bt x n.zip -o"/content/stable-diffusion-webui/embeddings"
  !rm n.zip
  clear()

# deepdanbooru и автозавершение тегов
def deepdanbooru():
  time.sleep(1.5)
  if booru_value == True:
    !git clone "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete.git" /content/stable-diffusion-webui/extensions/tag-autocomplete
    if not os.path.exists('/content/stable-diffusion-webui/tmp'):
      os.makedirs('/content/stable-diffusion-webui/tmp')
    tagacp = "/content/stable-diffusion-webui/tmp/tagAutocompletePath.txt"
    !touch $tagacp
    with open(tagacp, 'a') as ta:
      ta.write("extensions/a1111-sd-webui-tagcomplete/tags")
    !$wget $pt'deepdanbooru-v3-20211112-sgd-e28.zip' -P /content && unzip /content/deepdanbooru-v3-20211112-sgd-e28.zip  -d /content/stable-diffusion-webui/models/deepbooru/ && rm /content/deepdanbooru-v3-20211112-sgd-e28.zip
  if booru_value == False:
    None
  clear()

# скачивание VAE
def VAE():
  time.sleep(1.5)
  if not os.path.exists('/content/stable-diffusion-webui/models/VAE/'):
    os.makedirs('/content/stable-diffusion-webui/models/VAE/')
  vaemodels = [
  f"{dl} https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/VAE/vae-ft-mse.ckpt",
  f"{dl} https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -O /content/stable-diffusion-webui/models/VAE/vae-ft-ema.ckpt",
  f"{dl} {sf}kl-f8-trinart_characters.pt -O /content/stable-diffusion-webui/models/VAE/kl-f8-trinart_characters.ckpt",
  f"{dl} {sf}nivelai-vae.pt -O /content/stable-diffusion-webui/models/VAE/novelai-vae.ckpt",
  f"{dl} {sf}kl-f8-anime.pt -O /content/stable-diffusion-webui/models/VAE/kl-f8-anime.ckpt",
  f"{dl} {sf}kl-f8-anime2.pt -O /content/stable-diffusion-webui/models/VAE/kl-f8-anime2.ckpt",
  ]
  def vaemodels_inst(vaemodel):
    !$vaemodel
  with ThreadPoolExecutor(max_workers=len(vaemodels)) as executor:
    futures = [executor.submit(vaemodels_inst, vaemodel) for vaemodel in vaemodels]
    for future in as_completed(futures):
        future.result()
  clear()

# скачивание гиперсетей
def hypernets_install():
  time.sleep(1.5)
  hpath = "/content/stable-diffusion-webui/models/hypernetworks"
  dl = f"{wget} -P {hpath} "
  hypernetworks = [
        f"{dl} '{hyper}furry_2.pt'",
        f"{dl} '{hyper}aki99-112000.pt'",
        f"{dl} '{hyper}furry_protogen.pt'",
        f"{dl} '{hyper}anime.pt'",
        f"{dl} '{hyper}anime_2.pt'",
        f"{dl} '{hyper}aini.pt'",
        f"{dl} '{hyper}doyora-35000.pt'",
        f"{dl} '{hyper}furry_kemono.pt'",
        f"{dl} '{hyper}photo-100k.pt'",
        f"{dl} '{hyper}curssHN slime.pt'",
        f"{dl} '{hyper}nullmono-23000.pt'",
        f"{dl} '{hyper}newhp6.pt'",
        f"{dl} '{hyper}bangyou-hyper-77500.pt'",
        f"{dl} '{hyper}furry_scalie.pt'",
        f"{dl} '{hyper}furry_3.pt'",
        f"{dl} '{hyper}kyokucho-40000.pt'",
        f"{dl} '{hyper}furry_transformation.pt'",
        f"{dl} '{hyper}anime_3.pt'",
        f"{dl} '{hyper}molu-60000.pt'",
        f"{dl} '{hyper}furry.pt'",
        f"{dl} '{hyper}mumumu-hyper-61000.pt'",
        f"{dl} '{hyper}bigrbear-10000.pt'",
      ]
  def hypernetworks_inst(hypernetwork):
    !$hypernetwork
  with ThreadPoolExecutor(max_workers=len(hypernetworks)) as executor:
    futures = [executor.submit(hypernetworks_inst, hypernetwork) for hypernetwork in hypernetworks]
    for future in as_completed(futures):
        future.result()
    clear()
  if hypernet.value == False:
    None

# скачивание LORA
def LORA():
  time.sleep(1.5)
  if not os.path.exists('/content/stable-diffusion-webui/models/Lora'):
    os.makedirs('/content/stable-diffusion-webui/models/Lora')
  path = "/content/stable-diffusion-webui/models/Lora/"
  dl = f"{wget} -P {path} "
  loramodels = [
      f"{dl} {lora}Seiji_Shingen_idol-master.safetensors",
      f"{dl} {lora}PhotoVintage.pt",
      f"{dl} {lora}SilverjowStyle.safetensors",
      f"{dl} {lora}ArtisticJinskyStyle.safetensors",
      f"{dl} {lora}uncutPenis.safetensors",
      f"{dl} {lora}hoacb.safetensors",
      f"{dl} {lora}penguinFrontierStyle.safetensors",
      f"{dl} {lora}anime_penis.safetensors",
      f"{dl} {lora}priapusATarouMentaiko.safetensors",
      f"{dl} {lora}husbando.safetensors",
      f"{dl} {lora}jockstrap.safetensors",
      f"{dl} {lora}aenaluckStyle.safetensors",
      f"{dl} {lora}fConStyle.safetensors",
      f"{dl} {lora}wrestlingSinglets.safetensors",
      f"{dl} {lora}rnarccusStyle.safetensors",
      f"{dl} {lora}unionSuits.safetensors",
      f"{dl} {lora}pissDrinking.safetensors",
    ]
  def dwnld_lora(loramodel):
      !$loramodel
  with ThreadPoolExecutor(max_workers=len(loramodels)) as executor:
      futures = [executor.submit(dwnld_lora, loramodel) for loramodel in loramodels]
      for future in as_completed(futures):
          future.result()
  clear()

# дефолтные конфиги
def configs():
  time.sleep(1.5)
  ng() #нгрок
  defconfigs = [
      f"{dl_gh}localization.json -O /content/stable-diffusion-webui/localizations/localization.json",
      f"{dl_gh}config_noGD.json -O /content/stable-diffusion-webui/config.json",
      f"{dl_gh}ui-config.json -O /content/stable-diffusion-webui/ui-config.json",
    ]
  def defaultgonf(defconfig):
      !$defconfig
  with ThreadPoolExecutor(max_workers=len(defconfigs)) as executor:
      futures = [executor.submit(defaultgonf, defconfig) for defconfig in defconfigs]
      for future in as_completed(futures):
          future.result()
  clear()
  # сохранение картинок на гулодиск 
  if gdrive.value == True and os.path.exists("/content/gdrive/MyDrive"):
    with open("/content/stable-diffusion-webui/config.json") as r: text = r.read().replace('/content/output/', '/content/gdrive/MyDrive/SD/')
    with open("/content/stable-diffusion-webui/config.json", "w") as w: w.write(text)
    if os.path.exists("/content/gdrive/MyDrive/SD/styles.csv"):
      !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.csv.bak
      !ln -s /content/gdrive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
      !ln -s /content/gdrive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
      !chmod -c 777 /content/stable-diffusion-webui/styles.csv
      !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
    else:
      !rm /content/stable-diffusion-webui/styles.csv /content/stable-diffusion-webui/styles.bak
      !$dl_gh'styles.csv' -O /content/gdrive/MyDrive/SD/styles.csv 
      !ln -s /content/gdrive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv
      !ln -s /content/gdrive/MyDrive/SD/styles.csv /content/stable-diffusion-webui/styles.csv.bak
      !chmod -c 777 /content/stable-diffusion-webui/styles.csv
      !chmod -c 777 /content/stable-diffusion-webui/styles.csv.bak
  clear()
  if gdrive.value == False or (not os.path.exists("/content/gdrive/MyDrive")):
    None

# англоязычный интерфейс
def eng_lang():
  if eng.value == True:
    time.sleep(1.5)
    !rm /content/stable-diffusion-webui/localizations/localization.json
    with open("/content/stable-diffusion-webui/config.json") as r: text = r.read().replace('"localization": "localization",', '"localization": "",')
    with open("/content/stable-diffusion-webui/config.json", "w") as w: w.write(text)
  if eng.value == False:
    None

# кастомизация
def customisation():
  time.sleep(1.5)
  !rm /usr/local/lib/python3.8/dist-packages/gradio/strings.py
  !$dl_gh'gradio_strings.py' -O /usr/local/lib/python3.8/dist-packages/gradio/strings.py
  !$dl_gh'notification.mp3' -O /content/stable-diffusion-webui/notification.mp3
  !rm /content/stable-diffusion-webui/html/footer.html
  !$dl_gh'footer.html' -O /content/stable-diffusion-webui/html/footer.html
  !$dl_gh'user.css' -P /content/stable-diffusion-webui/
  c1 = "/content/stable-diffusion-webui/style.css"; c2 = "/content/stable-diffusion-webui/user.css"; open(c1, 'a+').write(open(c2, 'r').read()); open(c1).close(); open(c2).close()
  !rm /content/stable-diffusion-webui/user.css
  with open("/content/stable-diffusion-webui/modules/ui.py") as r: text = r.read().replace("print('ngrok authtoken detected, trying to connect...')", "")
  with open("/content/stable-diffusion-webui/modules/ui.py", "w") as w: w.write(text)
  with open("/content/stable-diffusion-webui/modules/sd_models.py") as r: text = r.read().replace('f"Checkpoint {model_checkpoint} not found; loading fallback {checkpoint_info.title}", file=sys.stderr', '"загрузка модели..."')
  with open("/content/stable-diffusion-webui/modules/sd_models.py", "w") as w: w.write(text)
  with open("/usr/local/lib/python3.8/dist-packages/gradio/blocks.py") as r: text = r.read().replace('                strings.en["RUNNING_LOCALLY_SEPARATED"].format(\n                    self.protocol, self.server_name, self.server_port\n                )', '                  ""')
  with open("/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", "w") as w: w.write(text)
  with open("/usr/local/lib/python3.8/dist-packages/gradio/blocks.py") as r: text = r.read().replace('strings.en["SHARE_LINK_DISPLAY"].format(self.share_url)', '"\033[01;38;05;112m⯈\033[0m Ссылка на Градио:", self.share_url')
  with open("/usr/local/lib/python3.8/dist-packages/gradio/blocks.py", "w") as w: w.write(text)
  with open("/content/stable-diffusion-webui/modules/ngrok.py") as r: text = r.read().replace("ngrok connected to localhost:{port}! URL: {public_url}\\n'\n               'You can use this link after the launch is complete.", "\\033[01;38;05;112m⯈\\033[0m Ссылка на Нгрок: {public_url}")
  with open("/content/stable-diffusion-webui/modules/ngrok.py", "w") as w: w.write(text)
  with open("/content/stable-diffusion-webui/modules/ngrok.py") as r: text = r.read().replace("f'Invalid ngrok authtoken, ngrok connection aborted.\\n'\n              f'Your token: {token}, get the right one on https://dashboard.ngrok.com/get-started/your-authtoken'", "'указан неверный токен или тебе выпал общий токен который использует другой анон, получи свежий: https://dashboard.ngrok.com/get-started/your-authtoken'")
  with open("/content/stable-diffusion-webui/modules/ngrok.py", "w") as w: w.write(text)
  # старый стиль для старой вебуи
  if checkpoint_value == "legacy":
    !rm /content/stable-diffusion-webui/style.css
    !$dl_gh'style.css'
  clear()

# удалить пустые папки .ipynb_checkpoints и мусор
def cleanup():
  time.sleep(1.5)
  !find /content/ -name ".ipynb_checkpoints" -type d -exec rm -rf {} \;
  sdwu = "/content/stable-diffusion-webui/"
  !rm -r {sdwu}webui.sh {sdwu}webui.bat {sdwu}webui-user.sh {sdwu}webui-user.bat {sdwu}webui-macos-env.sh {sdwu}screenshot.png {sdwu}README.md {sdwu}LICENSE.txt {sdwu}CODEOWNERS /content/sample_data

'''
def prerun():
  time.sleep(1.5)
  loader(inprogress)
  def pre_installation():
    if checkpoint_value == "legacy":
      if xformers_value == False:
        !python /content/stable-diffusion-webui/webui.py --disable-safe-unpickle --enable-insecure-extension-access
      if xformers_value == True:
        !python /content/stable-diffusion-webui/webui.py --xformers --disable-safe-unpickle --enable-insecure-extension-access
    if checkpoint_value == "new":
      if xformers_value == False:
        !python /content/stable-diffusion-webui/webui.py --disable-safe-unpickle --enable-insecure-extension-access --no-hashing
      if xformers_value == True:
        !python /content/stable-diffusion-webui/webui.py --xformers --disable-safe-unpickle --enable-insecure-extension-access --no-hashing
  if __name__ == '__main__':
    p = multiprocessing.Process(target=pre_installation)
    p.start()
    p.join(60)
    if p.is_alive():
      p.kill()
      p.join()
'''

def complite():
  clear()
  time.sleep(1.5)
  with capture.capture_output() as mem_allocate:
    !free -g | grep Mem | awk '{print $7}'
  with capture.capture_output() as mem_all:
    !free -g | grep Mem | awk '{print $2}'
  percent = round(int(str(mem_allocate)) / int(str(mem_all)) * 100, 2)
  display(HTML(f'<p class="setup_complite">Установка завершена<hr class="hr_setup_comlite"/><span class="memory">Доступно ОЗУ: <span class="mem_allocate">{mem_allocate} Гб</span> из <span class="mem_all">{mem_all} Гб</span> <span class="percent">({percent}% свободно)</span></span></p>'))
  if int(str(mem_allocate)) <= 7:
    display(HTML('<p class="little_memory">Осталось слишком мало памяти!<br/>Рекомендуется остановить ячейку и выполнить сброс памяти через ячейку <span class="cell_fix">фикс утечки памяти</span>, что находится выше</p>'))
  nsmi = getoutput('nvidia-smi')
  if "NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running." in nsmi:
    display(HTML(f'{gh}warn'))
  else:
    card = !nvidia-smi --query-gpu=gpu_name --format=csv
    vram = !nvidia-smi --query-gpu=memory.total --format=csv
    print("Видеокарта: \033[01;38;05;46mNvidia " + (card[1]) + "\033[0m |" + "\033[01;38;05;188m Объём видеопамяти: \033[01;38;05;46m" + (vram[1]) + "\033[0m ")
  cou.eval_js(f'new Audio("{gh}finish.mp3").play()')

def run_webui():
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  if starton_value == True:
    !sleep 3
    cou.eval_js(f'new Audio("{gh}start.mp3").play()')
    !sleep 3
    #Установка токена ngrok
    ngrok_share = cfg+'cfg'
    freetoken = open(ngrok_share).read().split('\n')
    sharetoken = random.choice(list(freetoken))
    token = ngrok_token
    if not ngrok_token_value or len(ngrok_token_value) < 48 or len(ngrok_token_value) > 51:
      ngrok = f'--ngrok {sharetoken} --ngrok-region eu'
    else:
      ngrok = f'--ngrok {ngrok_token_value} --ngrok-region eu'
    if xformers_value == True:
      xf = '--xformers'
    if xformers_value == False:
      xf = '--opt-split-attention'
    if no_half_vae_value == True:
      nhv = '--no-half-vae'
    if no_half_vae_value == False:
      nhv = ''
    if checkpoint_value == "new":
      gq = '--gradio-queue --disable-console-progressbars --no-hashing'
    if checkpoint_value == "legacy":
      gq = ''
    ckpt_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"
    if booru_value == True:
      deepdanbooru_arg = "--deepdanbooru"
    if booru_value == False:
      deepdanbooru_arg = ""
    default_args = f"--disable-safe-unpickle --enable-insecure-extension-access --share {deepdanbooru_arg} --gradio-img2img-tool editor --theme dark --ckpt-dir {ckpt_dir}"
    cmdargs = f'{default_args} {gq} {xf} {nhv} {ngrok}'
    #with capture.capture_output() as stored_variables:
    #  %store cmdargs
    saved_vars = open(f'{cfg}saved_vars', 'wb')
    pickle.dump(xf, saved_vars)
    pickle.dump(nhv, saved_vars)
    pickle.dump(gq, saved_vars)
    pickle.dump(deepdanbooru_arg, saved_vars)
    pickle.dump(default_args, saved_vars)
    saved_vars.close()
    print("\033[1;36mВнимание! Если интерфейсе не отображается генерация, просто используй ссылку на Gradio!\033[0m \n")
    print("\033[1;34mПереходить по ссылкам можно после полной прогрузки, ожидай сообщения.\033[0m")
    flara()
    with open('/content/stable-diffusion-webui/clfr.txt', 'r') as flare:
      print("\033[01;38;05;112m⯈\033[0m Ссылка на Флару: " + flare.read())
    #from google.colab.output import serve_kernel_port_as_window
    #from google.colab.output import eval_js
    #print("\033[01;38;05;112m⯈\033[0m Ссылка на Гугл:", eval_js( "google.colab.kernel.proxyPort(" + str(7860) + ")" ))
    print("")
    #Собственно запуск
    # в последней версии автоматик добавил аргументы: --no-hashing, можно использовать вместе с --lowram
    !python /content/stable-diffusion-webui/webui.py $cmdargs
    if os.path.exists("/content/gdrive/MyDrive/SD/styles.csv"):
      !cp -f /content/stable-diffusion-webui/styles.csv /content/gdrive/MyDrive/SD/styles.csv
    display(HTML('/content/.config/end'))
    if starton_value == False:
      None


# фикс жора ОЗУ
def ram_liberator():
  debs = [
    "wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb",
    "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb",
    "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb",
    "wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb",
    "apt install -qq libunwind8-dev",
    ]
  def colab_downgrade(deb):
    !$deb
  with ThreadPoolExecutor(max_workers=len(debs)) as executor:
    futures = [executor.submit(colab_downgrade, deb) for deb in debs]
    for future in as_completed(futures):
      future.result()
  !dpkg -i *.deb && rm  *.deb
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
  clear()

#ver 19.02

#       lambda: on_download_click(None),
def parallels_tasks():
    tasks = [
        extentions_builtin,
        extentions_install,
        extentions_mix_install,
        upscalers,
        emdeddings_install,
        emdeddings_for_negative,
        LORA,
        VAE,
        hypernets_install,
        deepdanbooru,
        instruct_pix2pix,
        ControlNet_install,
        misc_models,
        interrogate_clip_install(),
    ]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor: #временно 16 вместо (max_workers=len(tasks)) чтобы не задыхалось
        # шлем каждый task в отдельный executor
        futures = [executor.submit(run_task, task) for task in tasks]
        for future in concurrent.futures.as_completed(futures):
            # ждем завершения, выплевываем ошибки
            try:
                future.result()
            except Exception as e:
                print(f"Error in task: {e}")

def run_task(task):
    # Новый ThreadPoolExecutor для каждого task
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # запускаем каждый task в новом executor
        executor.submit(task)

def store_values_external(): # тоже самое через внешний файл
  time.sleep(1.5)
  !$dl_gh'store_values.ipynb' -P $cfg
  %run $cfg'store_values.ipynb'

# все действия после нажатия на кнопку "продолжить"
def apply_button_click(b): # выполнение последрвательно всего выше (паузы time.sleep(1.5) внутри самих def) #prerun()
  store_values_external()
  ram_liberator()
  depinst()
  A1111_webui_install()
  ultimate_gdrive_symlink()
  parallels_tasks()
  on_download_click(None)
  configs()
  eng_lang()
  customisation()
  mobile_player()
  lora_to_extention()
  cleanup()
  complite()
  run_webui()

